In [ ]:
# To do
# Test out variable layers
# Add regularization
# Add hyper-parameter tuning
# Check everything
# Run

In [6]:
# Install idx2numpy package for extracting data
!pip install idx2numpy

  Created wheel for idx2numpy: filename=idx2numpy-1.2.3-cp36-none-any.whl size=7905 sha256=6b0a91a3795f6a677eb5ba1236bb254fdd5c094547eda9b7c328d278d99e7478
  Stored in directory: /root/.cache/pip/wheels/7a/c1/da/284ce80a748fab898b8d1fa95468a386e7cf3b81da18511f9d
Successfully built idx2numpy


In [7]:
# Import packages
import gzip
import torch
import torchvision
import numpy as np 

import idx2numpy
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [8]:
# Mount Google drive to access data from Colab
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [9]:
def load_one_dataset(path):
    
    f = gzip.open(path, 'rb')
    data = torch.from_numpy(idx2numpy.convert_from_file(f))
    f.close()
    
    return(data)


def load_all_datasets(train_imgs, train_labs, test_imgs, test_labs, batch_size):
    
    
    train_images = load_one_dataset(train_imgs).type(torch.float32)
    train_labels = load_one_dataset(train_labs).type(torch.long)
    train = list(zip(train_images, train_labels))
    
    test_images = load_one_dataset(test_imgs).type(torch.float32)
    test_labels = load_one_dataset(test_labs).type(torch.long)
    test = list(zip(test_images, test_labels))
    
    train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True, num_workers=2)
    test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=False, num_workers=2)
    
    return(train_loader, test_loader)

In [ ]:
class Net(nn.Module):
  
  def __init__(self, nb_layers, nb_units, input_dim, output_dim):
    '''
    Layers are constructed here
    '''
    super(DNN, self).__init__()
    self.nb_layers = nb_layers
    fc = []
    
    for i in range(self.nb_layers):
        if i == 0:
            fc.append(nn.Linear(input_dim, nb_units[i]))
        elif i == nb_layers-1:
            fc.append(nn.Linear(nb_units[i], output_dim))
        else:
            fc.append(nn.Linear(nb_units[i], nb_units))
    
    self.fc = nn.ModuleList(fc)
  
  def forward(self, x):
    '''
    Forward propagation happens here
    '''
    for i in range(self.nb_layers):
      x = F.relu(self.fc[i](x))
    
    return x

In [11]:
def train(nb_units=[256, 128, 64, 32, 16], input_dim=784, output_dim = 10, 
          epochs=2, lr=0.001, momentum=0.9, batch_size=512):
    
    
    # Set paths to datasets
    paths = {
        
        'train_imgs': '/content/gdrive/MyDrive/data/train-images-idx3-ubyte.gz',
        'train_labs': '/content/gdrive/MyDrive/data/train-labels-idx1-ubyte.gz',
        'test_imgs': '/content/gdrive/MyDrive/data/t10k-images-idx3-ubyte.gz',
        'test_labs': '/content/gdrive/MyDrive/data/t10k-labels-idx1-ubyte.gz'
    }
    
    # Load datasets
    train_loader, test_loader = load_all_datasets(**paths, batch_size = batch_size)
    
    # Set parameters
    net = Net()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=lr, momentum=momentum)
    
    # Loop over the dataset multiple times
    for epoch in range(epochs):  
        
        # Initialize running loss
        running_loss = 0.0
        
        # Iterate through data now
        for i, data in enumerate(train_loader):
            
            # Get the inputs: data is a list of [inputs, labels]
            inputs, labels = data

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward
            outputs = net(inputs)
            
            # Calculate loss
            loss = criterion(outputs, labels)
            
            # Backward
            loss.backward()
            
            # Optimize
            optimizer.step()

            # Print statistics
            running_loss += loss.item()
        
        # Initialize the validation running loss
        val_running_loss = 0.0
        
        # No need to calculate gradients for validation set
        with torch.no_grad():

            # Loop through the validation data
            for i, data in enumerate(test_loader):
              
              # Get the data item 
              val_inputs, val_labels = data

              # Send the data item through the network to get output
              val_outputs = net(val_inputs)

              # Compute the loss
              val_loss = criterion(val_outputs, val_labels)

              # Add to the running validation loss
              val_running_loss += val_loss.item()
            
        # Print every 100 mini-batches
        print("The train loss on epoch {} is {}...".format(epoch, running_loss))
        print("The validation loss on epoch {} is {}...".format(epoch, val_running_loss))
    
    # Print message
    print('Done training...')

In [12]:
train(epochs=100)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  after removing the cwd from sys.path.


The train loss on epoch 0 is 417.0383608341217...
The validation loss on epoch 0 is 46.05197238922119...
The train loss on epoch 1 is 271.70668840408325...
The validation loss on epoch 1 is 46.05197238922119...
The train loss on epoch 2 is 271.70668840408325...
The validation loss on epoch 2 is 46.05197238922119...
The train loss on epoch 3 is 271.70668840408325...
The validation loss on epoch 3 is 46.05197238922119...
The train loss on epoch 4 is 271.70668840408325...
The validation loss on epoch 4 is 46.05197238922119...
The train loss on epoch 5 is 271.70668840408325...
The validation loss on epoch 5 is 46.05197238922119...
The train loss on epoch 6 is 271.70668840408325...
The validation loss on epoch 6 is 46.05197238922119...
The train loss on epoch 7 is 271.70668840408325...
The validation loss on epoch 7 is 46.05197238922119...
The train loss on epoch 8 is 271.70668840408325...
The validation loss on epoch 8 is 46.05197238922119...
The train loss on epoch 9 is 271.70668840408325